In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set parameters
np.random.seed(42)
days = 365
simulations = 1000  # Number of simulations
initial_price = 50  # Initial price in EUR/MWh
mu = 0.0005  # Drift (expected daily return)
sigma = 0.02  # Volatility (daily standard deviation of returns)

# Monte Carlo simulation for multiple price paths
def monte_carlo_simulation(initial_price, mu, sigma, days, simulations):
    price_matrix = np.zeros((simulations, days))
    price_matrix[:, 0] = initial_price

    for t in range(1, days):
        # Generate daily returns based on normal distribution (drift + volatility)
        random_returns = np.random.normal(mu, sigma, simulations)
        price_matrix[:, t] = price_matrix[:, t - 1] * np.exp(random_returns)
    
    return price_matrix

# Run the simulation
price_paths = monte_carlo_simulation(initial_price, mu, sigma, days, simulations)

# Create a DataFrame for plotting
dates = pd.date_range(start='2024-01-01', periods=days)
df_simulations = pd.DataFrame(price_paths.T, index=dates)

# Plot a few random simulations (for visualization purposes)
plt.figure(figsize=(10, 6))
for i in range(simulations):  # Plot only 10 random simulations for clarity
    plt.plot(df_simulations.index, df_simulations.iloc[:, i], lw=1)
plt.title(f'Monte Carlo Simulated Electricity Price Paths ({simulations} Simulations)')
plt.xlabel('Date')
plt.ylabel('Price (EUR/MWh)')
plt.grid(True)
plt.show()

# Summary statistics for the final day prices
final_prices = df_simulations.iloc[-1, :]
mean_price = np.mean(final_prices)
median_price = np.median(final_prices)
std_price = np.std(final_prices)

print(f"Mean final price: {mean_price:.2f} EUR/MWh")
print(f"Median final price: {median_price:.2f} EUR/MWh")
print(f"Standard deviation of final price: {std_price:.2f} EUR/MWh")
